# OG Data

In [2]:
import os
import glob

In [3]:
dat0 = '/root/dataset/'
litedat0 = '/root/datasetlite/'
dat1 = '/*.tfrecords'

In [4]:
#regular data
train_data = glob.glob(dat0+'train'+dat1)
test_1_data = glob.glob(dat0+'test-1'+dat1)
test_2_data = glob.glob(dat0+'test-2'+dat1)
validate_data = glob.glob(dat0+'validate'+dat1)
validate2_data = glob.glob(dat0+'validate-2'+dat1)

#lite data
lite_train_data = glob.glob(litedat0+'train'+dat1)
lite_test_1_data = glob.glob(litedat0+'test-1'+dat1)
lite_test_2_data = glob.glob(litedat0+'test-2'+dat1)
lite_validate_data = glob.glob(litedat0+'validate'+dat1)
lite_validate2_data = glob.glob(litedat0+'validate-2'+dat1)

# New Data

In [5]:
# import pandas as pd
# import numpy as np

In [6]:
abidata = '/root/abidata/digi_twin_summer_wireless_dataset/allTrials'

In [7]:
# pd.read_csv(abidata+'/trial-1116731144_kpis.txt',header=None).loc[0]

In [8]:

# t_sample = np.resize(pd.read_csv(abidata+'/trial-1116731144_traffic.txt',header=None).loc[0], (10, 2))

# Models

In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

## Old Model

In [10]:
# old model
# class RouteNet(tf.keras.Model):
#     def __init__(self,hparams, output_units=1, final_activation=None):
#         super(RouteNet, self).__init__()

#         self.hparams = hparams
#         self.output_units = output_units
#         self.final_activation = final_activation
        

        
#     def build(self, input_shape=None):
#         del input_shape

#         self.edge_update = tf.keras.layers.GRUCell(self.hparams.link_state_dim, name="edge_update")
#         self.path_update = tf.keras.layers.GRUCell(self.hparams.path_state_dim, name="path_update")
        
        
#         self.readout = tf.keras.models.Sequential(name='readout')

#         for i in range(self.hparams.readout_layers):
#             self.readout.add(tf.keras.layers.Dense(self.hparams.readout_units, 
#                     activation=tf.nn.selu,
#                     kernel_regularizer=tf.contrib.layers.l2_regularizer(self.hparams.l2)))
#             self.readout.add(tf.keras.layers.Dropout(rate=self.hparams.dropout_rate))

#         self.final = keras.layers.Dense(self.output_units, 
#                 kernel_regularizer=tf.contrib.layers.l2_regularizer(self.hparams.l2_2),
#                 activation = self.final_activation )
        
#         self.edge_update.build(tf.TensorShape([None,self.hparams.path_state_dim]))
#         self.path_update.build(tf.TensorShape([None,self.hparams.link_state_dim]))
#         self.readout.build(input_shape = [None,self.hparams.path_state_dim])
#         self.final.build(input_shape = [None,self.hparams.path_state_dim + self.hparams.readout_units ])

#         self.mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

#         self.built = True
    
    
#     def call(self, inputs, training=False):
#         '''

#         outputs:
#             Natural parameter
        
#         '''
#         #this takes features
#         f_ = inputs
#         sample = inputs

#         shape = tf.stack([f_['n_links'],self.hparams.link_state_dim-1], axis=0)
#         #link_state = tf.zeros(shape)
#         link_state = tf.concat([
#             tf.expand_dims(f_['capacities'],axis=1),
#             tf.zeros(shape)
#         ], axis=1)

#         shape = tf.stack([f_['n_paths'],self.hparams.path_state_dim-1], axis=0)
#         path_state = tf.concat([
#             tf.expand_dims(f_['traffic'][0:f_["n_paths"]],axis=1),
#             tf.zeros(shape)
#         ], axis=1)

#         links = f_['links']
#         paths = f_['paths']
#         seqs=  f_['sequences']
        
#         for _ in range(self.hparams.T):
        
#             h_ = tf.gather(link_state,links)

#             #TODO move this to feature calculation
#             ids=tf.stack([paths, seqs], axis=1)            
#             max_len = tf.reduce_max(seqs)+1
#             shape = tf.stack([f_['n_paths'], max_len, self.hparams.link_state_dim])
#             lens = tf.segment_sum(data=tf.ones_like(paths),
#                                     segment_ids=paths)

#             link_inputs = tf.scatter_nd(ids, h_, shape)
#             #TODO move to tf.keras.RNN
#             outputs, path_state = tf.nn.dynamic_rnn(self.path_update,
#                                                     link_inputs,
#                                                     sequence_length=lens,
#                                                     initial_state = path_state,
#                                                     dtype=tf.float32)
#             m = tf.gather_nd(outputs,ids)

#             m = tf.unsorted_segment_sum(m, links ,f_['n_links'])

#             #Keras cell expects a list
#             link_state,_ = self.edge_update(m, [link_state])
            
#         if self.hparams.learn_embedding:
#             r = self.readout(path_state,training=training)
#             o = self.final(tf.concat([r,path_state], axis=1))
            
#         else:
#             r = self.readout(tf.stop_gradient(path_state),training=training)
#             o = self.final(tf.concat([r, tf.stop_gradient(path_state)], axis=1) )

#         return o

# New Model

In [11]:
# class CustomModel(tf.keras.Sequential):
#   """A custom sequential model that overrides `Model.train_step`."""

#   def train_step(self, data):
#     batch_data, labels = data

#     with tf.GradientTape() as tape:
#       predictions = self(batch_data, training=True)
#       # Compute the loss value (the loss function is configured
#       # in `Model.compile`).
#       loss = self.compiled_loss(labels, predictions)

#     # Compute the gradients of the parameters with respect to the loss.
#     gradients = tape.gradient(loss, self.trainable_variables)
#     # Perform gradient descent by updating the weights/parameters.
#     self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
#     # Update the metrics (includes the metric that tracks the loss).
#     self.compiled_metrics.update_state(labels, predictions)
#     # Return a dict mapping metric names to the current values.
#     return {m.name: m.result() for m in self.metrics}

In [41]:
class RouteNet(tf.keras.Model):
    def __init__(self,hparams, output_units=1, final_activation=None):
        super(RouteNet, self).__init__()

        self.hparams = hparams
        self.output_units = output_units
        self.final_activation = final_activation
        

        
    def build(self, input_shape=None):
        del input_shape

        self.edge_update = tf.keras.layers.GRUCell(self.hparams.link_state_dim, name="edge_update")
        self.path_update = tf.keras.layers.GRUCell(self.hparams.path_state_dim, name="path_update")
        
        
        self.readout = tf.keras.models.Sequential(name='readout')

        for i in range(self.hparams.readout_layers):
            self.readout.add(tf.keras.layers.Dense(self.hparams.readout_units, 
                    activation=tf.nn.selu,
                    kernel_regularizer=tf.contrib.layers.l2_regularizer(self.hparams.l2)))
            self.readout.add(tf.keras.layers.Dropout(rate=self.hparams.dropout_rate))

        self.final = keras.layers.Dense(self.output_units, 
                kernel_regularizer=tf.contrib.layers.l2_regularizer(self.hparams.l2_2),
                activation = self.final_activation )
        
        self.edge_update.build(tf.TensorShape([None,self.hparams.path_state_dim]))
        self.path_update.build(tf.TensorShape([None,self.hparams.link_state_dim]))
        self.readout.build(input_shape = [None,self.hparams.path_state_dim])
        self.final.build(input_shape = [None,self.hparams.path_state_dim + self.hparams.readout_units ])

        self.mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

        self.built = True
    
    
    def call(self, inputs, training=False):
        '''

        outputs:
            Natural parameter
        
        '''
        #this takes features
        f_ = inputs

        shape = tf.stack([f_[6],self.hparams.link_state_dim-1], axis=0)
        #link_state = tf.zeros(shape)
        link_state = tf.concat([
            tf.expand_dims(f_[1],axis=1),
            tf.zeros(shape)
        ], axis=1)

        shape = tf.stack([f_[7],self.hparams.path_state_dim-1], axis=0)
        path_state = tf.concat([
            tf.expand_dims(f_[0][0:f_[7]],axis=1),
            tf.zeros(shape)
        ], axis=1)

        links = f_[4]
        paths = f_[5]
        seqs=  f_[2]
        
        for _ in range(self.hparams.T):
        
            h_ = tf.gather(link_state,links)

            #TODO move this to feature calculation
            ids=tf.stack([paths, seqs], axis=1)            
            max_len = tf.reduce_max(seqs)+1
            shape = tf.stack([f_[7], max_len, self.hparams.link_state_dim])
            lens = tf.segment_sum(data=tf.ones_like(paths),
                                    segment_ids=paths)

            link_inputs = tf.scatter_nd(ids, h_, shape)
            #TODO move to tf.keras.RNN
            outputs, path_state = tf.nn.dynamic_rnn(self.path_update,
                                                    link_inputs,
                                                    sequence_length=lens,
                                                    initial_state = path_state,
                                                    dtype=tf.float32)
            m = tf.gather_nd(outputs,ids)

            m = tf.unsorted_segment_sum(m, links ,f_[6])

            #Keras cell expects a list
            link_state,_ = self.edge_update(m, [link_state])
            
        if self.hparams.learn_embedding:
            r = self.readout(path_state,training=training)
            o = self.final(tf.concat([r,path_state], axis=1))
            
        else:
            r = self.readout(tf.stop_gradient(path_state),training=training)
            o = self.final(tf.concat([r, tf.stop_gradient(path_state)], axis=1) )

        return o


    def train_step(self, data):
        features, labels = data
        
        with tf.GradientTape() as tape:
            predictions = self(features, training=True)
            loc  = predictions[...,0] 
            c = np.log(np.expm1( np.float32(0.098) ))
            scale = tf.math.softplus(c + predictions[...,1]) + np.float32(1e-9)
            delay_prediction = loc
            jitter_prediction = scale**2
            
            x = features
            y = labels
            n = x[3] - y[2]
            _2sigma = np.float32(2.0)*scale**2
            nll = n*y[3]/_2sigma + n*tf.math.squared_difference(y[0], loc)/_2sigma + n*tf.math.log(scale)
            loss = tf.reduce_sum(nll)/np.float32(1e6)
            regularization_loss = sum(self.losses)
            total_loss = loss + regularization_loss

        gradients = tape.gradient(total_loss, self.trainable_variables)

        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        self.mae_metric.update_state(labels, predictions)

        return {m.name: m.result() for m in self.metrics}

In [13]:
# class AbishekNet(tf.keras.Model):
#     def __init__(self,hparams, output_units=2, final_activation=None):
#         super(RouteNet, self).__init__()

#         self.hparams = hparams
#         self.output_units = output_units
#         self.final_activation = final_activation
        

        
#     def build(self, input_shape=None):
#         del input_shape

#         self.edge_update = tf.keras.layers.GRUCell(self.hparams.link_state_dim, name="edge_update")
#         self.path_update = tf.keras.layers.GRUCell(self.hparams.path_state_dim, name="path_update")
        
        
#         self.readout = tf.keras.models.Sequential(name='readout')

#         for i in range(self.hparams.readout_layers):
#             self.readout.add(tf.keras.layers.Dense(self.hparams.readout_units, 
#                     activation=tf.nn.selu,
#                     kernel_regularizer=tf.contrib.layers.l2_regularizer(self.hparams.l2)))
#             self.readout.add(tf.keras.layers.Dropout(rate=self.hparams.dropout_rate))

#         self.final = keras.layers.Dense(self.output_units, 
#                 kernel_regularizer=tf.contrib.layers.l2_regularizer(self.hparams.l2_2),
#                 activation = self.final_activation )
        
#         self.edge_update.build(tf.TensorShape([None,self.hparams.path_state_dim]))
#         self.path_update.build(tf.TensorShape([None,self.hparams.link_state_dim]))
#         self.readout.build(input_shape = [None,self.hparams.path_state_dim])
#         self.final.build(input_shape = [None,self.hparams.path_state_dim + self.hparams.readout_units ])

#         self.mae_metric = keras.metrics.MeanAbsoluteError(name="mae")

#         self.built = True
    
    
#     def call(self, inputs, training=False):
#         '''

#         outputs:
#             Natural parameter
        
#         '''
#         #this takes features
#         f_ = inputs
#         sample = inputs


#         n_links = 20

#         shape = tf.stack([n_links,self.hparams.link_state_dim-1], axis=0)
#         link_state = tf.zeros(shape)
#         # link_state = tf.concat([
#         #     tf.expand_dims(f_['capacities'],axis=1),
#         #     tf.zeros(shape)
#         # ], axis=1)

#         n_paths = 10
#         shape = tf.stack([n_paths,self.hparams.path_state_dim-1], axis=0)
#         # path_state = tf.concat([
#         #     tf.expand_dims(f_['traffic'][0:f_["n_paths"]],axis=1),
#         #     tf.zeros(shape)
#         # ], axis=1)
#         path_state = tf.zeros(shape)
#         links = f_['links']
#         paths = f_['paths']
#         seqs = f_['sequences']
        
#         for _ in range(self.hparams.T):
        
#             h_ = tf.gather(link_state,links)

#             #TODO move this to feature calculation
#             ids=tf.stack([paths, seqs], axis=1)            
#             max_len = tf.reduce_max(seqs)+1

#             shape = tf.stack([n_paths, max_len, self.hparams.link_state_dim])
#             lens = tf.segment_sum(data=tf.ones_like(paths),
#                                     segment_ids=paths)

#             link_inputs = tf.scatter_nd(ids, h_, shape)
#             #TODO move to tf.keras.RNN
#             outputs, path_state = tf.nn.dynamic_rnn(self.path_update,
#                                                     link_inputs,
#                                                     sequence_length=lens,
#                                                     initial_state = path_state,
#                                                     dtype=tf.float32)
#             m = tf.gather_nd(outputs,ids)
#             m = tf.unsorted_segment_sum(m, links ,n_links)

#             #Keras cell expects a list
#             link_state,_ = self.edge_update(m, [link_state])
            
#         if self.hparams.learn_embedding:
#             r = self.readout(path_state,training=training)
#             o = self.final(tf.concat([r,path_state], axis=1))
            
#         else:
#             r = self.readout(tf.stop_gradient(path_state),training=training)
#             o = self.final(tf.concat([r, tf.stop_gradient(path_state)], axis=1) )

#         return o


#     def train_step(self, data):
#         features, labels = data
#         with tf.GradientTape() as tape:
#             predictions = self(features, training=True)
#             loc  = predictions[...,0] 
#             c = np.log(np.expm1( np.float32(0.098) ))
#             scale = tf.math.softplus(c + predictions[...,1]) + np.float32(1e-9)
#             delay_prediction = loc
#             jitter_prediction = scale**2
            
#             x = features
#             y = labels
#             packets = y[:, 0]
#             drops = y[:,1]
#             n = packets-drops
#             # n = x['packets']-y['drops']
#             _2sigma = np.float32(2.0)*scale**2
            
#             delay = y[:, 3]
#             nll = n*tf.math.squared_difference(delay, loc)/_2sigma + n*tf.math.log(scale)
#             # nll = n*y['jitter']/_2sigma + n*tf.math.squared_difference(y['delay'], loc)/_2sigma + n*tf.math.log(scale)
            
#             loss = tf.reduce_sum(nll)/np.float32(1e6)
#             regularization_loss = sum(self.losses)
#             total_loss = loss + regularization_loss

#         gradients = tape.gradient(total_loss, self.trainable_variables)

#         self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

#         self.mae_metric(labels, predictions)

#         return {m.name: m.result() for m in self.metrics}

# Old Data Pipeline

In [14]:
def scale_fn(k, val):
    '''Scales given feature
    Args:
        k: key
        val: tensor value
    '''

    if k == 'traffic':
        return (val-0.18)/.15
    if k == 'capacities':
        return val/10.0

    return val

In [15]:
# def scale_fn(k, val):
#     '''Scales given feature
#     Args:
#         k: key
#         val: tensor value
#     '''

#     if k == 0:
#         return (val-0.18)/.15
#     if k == 6:
#         return val/10.0

#     return val

In [16]:
def parse(serialized, target=None, normalize=True):
    '''
    Target is the name of predicted variable-deprecated
    '''
    with tf.device("/cpu:0"):
        with tf.name_scope('parse'):    
            #TODO add feature spec class
            features = tf.io.parse_single_example(
                serialized,
                features={
                    'traffic':tf.VarLenFeature(tf.float32),
                    'delay':tf.VarLenFeature(tf.float32),
                    'logdelay':tf.VarLenFeature(tf.float32),
                    'jitter':tf.VarLenFeature(tf.float32),
                    'drops':tf.VarLenFeature(tf.float32),
                    'packets':tf.VarLenFeature(tf.float32),
                    'capacities':tf.VarLenFeature(tf.float32),
                    'links':tf.VarLenFeature(tf.int64),
                    'paths':tf.VarLenFeature(tf.int64),
                    'sequences':tf.VarLenFeature(tf.int64),
                    'n_links':tf.FixedLenFeature([],tf.int64), 
                    'n_paths':tf.FixedLenFeature([],tf.int64),
                    'n_total':tf.FixedLenFeature([],tf.int64)
                })
            for k in ['traffic','delay','logdelay','jitter','drops','packets','capacities','links','paths','sequences']:
                features[k] = tf.sparse.to_dense( features[k] )
                if normalize:
                    features[k] = scale_fn(k, features[k])


    #return {k:v for k,v in features.items() if k is not target },features[target]
    return features

In [17]:
# def parse(serialized, target=None, normalize=True):
#     '''
#     Target is the name of predicted variable-deprecated
#     '''
#     with tf.device("/cpu:0"):
#         with tf.name_scope('parse'):    
#             #TODO add feature spec class
#             features = tf.io.parse_single_example(
#                 serialized,
#                 features={
#                     'traffic':tf.VarLenFeature(tf.float32),#0 traffic
#                     'delay':tf.VarLenFeature(tf.float32),#1 delay
#                     'logdelay':tf.VarLenFeature(tf.float32),#2 logdelay
#                     'jitter':tf.VarLenFeature(tf.float32),#3 jitter
#                     'drops':tf.VarLenFeature(tf.float32),#4 drops
#                     'packets':tf.VarLenFeature(tf.float32),#5 packets
#                     'capacities':tf.VarLenFeature(tf.float32),#6 capacities
#                     'links':tf.VarLenFeature(tf.int64),#7 links
#                     'paths':tf.VarLenFeature(tf.int64),#8 paths
#                     'sequences':tf.VarLenFeature(tf.int64),#9 sequences
#                     'n_links':tf.FixedLenFeature([],tf.int64), #10 n_links
#                     'n_paths':tf.FixedLenFeature([],tf.int64), #11 n_paths
#                     'n_total':tf.FixedLenFeature([],tf.int64) #12 n_total
#                 })
#             features = [features[f] for f in features.keys]
#             for k in [0,1,2,3,4,5,6,7,8,9]:
#                 features[k] = tf.sparse.to_dense( features[k] )
#                 if normalize:
#                     features[k] = scale_fn(k, features[k])


#     #return {k:v for k,v in features.items() if k is not target },features[target]
#     return features

In [18]:
def cummax(alist, extractor):
    with tf.name_scope('cummax'):
        maxes = [tf.reduce_max( extractor(v) ) + 1 for v in alist ]
        cummaxes = [tf.zeros_like(maxes[0])]
        for i in range(len(maxes)-1):
            cummaxes.append( tf.math.add_n(maxes[0:i+1]))
        
    return cummaxes

In [19]:
# def transformation_func(it, batch_size=32):
#     with tf.name_scope("transformation_func"):
#         vs = [it.get_next() for _ in range(batch_size)]
        
#         links_cummax = cummax(vs,lambda v:v['links'] )
#         paths_cummax = cummax(vs,lambda v:v['paths'] )
        
#         tensors = ({
#                 'traffic':tf.concat([v['traffic'] for v in vs], axis=0),
#                 'capacities': tf.concat([v['capacities'] for v in vs], axis=0),
#                 'sequences':tf.concat([v['sequences'] for v in vs], axis=0),
#                 'packets':tf.concat([v['packets'] for v in vs], axis=0),
#                 'links':tf.concat([v['links'] + m for v,m in zip(vs, links_cummax) ], axis=0),
#                 'paths':tf.concat([v['paths'] + m for v,m in zip(vs, paths_cummax) ], axis=0),
#                 'n_links':tf.math.add_n([v['n_links'] for v in vs]),
#                 'n_paths':tf.math.add_n([v['n_paths'] for v in vs]),
#                 'n_total':tf.math.add_n([v['n_total'] for v in vs])
#             },   {
#                 'delay' : tf.concat([v['delay'] for v in vs], axis=0),
#                 'logdelay' : tf.concat([v['logdelay'] for v in vs], axis=0),
#                 'drops' : tf.concat([v['drops'] for v in vs], axis=0),
#                 'jitter' : tf.concat([v['jitter'] for v in vs], axis=0),
#                 }
#             )
    
#     return tensors

In [20]:
# def transformation_func(it, batch_size=32):
#     with tf.name_scope("transformation_func"):
#         vs = [it.get_next() for _ in range(batch_size)]
        
#         links_cummax = cummax(vs,lambda v:v[7] )
#         paths_cummax = cummax(vs,lambda v:v[8] )
        
#         tensors = ([
#                 tf.concat([v[0] for v in vs], axis=0),#0 traffic
#                 tf.concat([v[6] for v in vs], axis=0),#1 capacities
#                 tf.concat([v[9] for v in vs], axis=0),#2 sequences
#                 tf.concat([v[5] for v in vs], axis=0),#3 packets
#                 tf.concat([v[7] + m for v,m in zip(vs, links_cummax) ], axis=0),#4 links
#                 tf.concat([v[8] + m for v,m in zip(vs, paths_cummax) ], axis=0),#5 paths
#                 tf.math.add_n([v[10] for v in vs]),#6 n_links
#                 tf.math.add_n([v[11] for v in vs]),#7 n_paths
#                 tf.math.add_n([v[12] for v in vs])#8 n_total
#             ],   [
#                 tf.concat([v[1] for v in vs], axis=0),#0 delay
#                 tf.concat([v[2] for v in vs], axis=0),#1 logdelay
#                 tf.concat([v[4] for v in vs], axis=0),#2 drops
#                 tf.concat([v[3] for v in vs], axis=0),#3 jitter
#                 ]
#             )
    
#     return tensors

In [21]:
def transformation_func(it, batch_size=32):
    with tf.name_scope("transformation_func"):
        vs = [it.get_next() for _ in range(batch_size)]
        
        links_cummax = cummax(vs,lambda v:v['links'] )
        paths_cummax = cummax(vs,lambda v:v['paths'] )
        
        tensors = ([
                tf.concat([v['traffic'] for v in vs], axis=0),#0 traffic
                tf.concat([v['capacities'] for v in vs], axis=0),#1 capacities
                tf.concat([v['sequences'] for v in vs], axis=0),#2 sequences
                tf.concat([v['packets'] for v in vs], axis=0),#3 packets
                tf.concat([v['links'] + m for v,m in zip(vs, links_cummax) ], axis=0),#4 links
                tf.concat([v['paths'] + m for v,m in zip(vs, paths_cummax) ], axis=0),#5 paths
                tf.math.add_n([v['n_links'] for v in vs]),#6 n_links
                tf.math.add_n([v['n_paths'] for v in vs]),#7 n_paths
                tf.math.add_n([v['n_total'] for v in vs])#8 n_total
            ],   [
                tf.concat([v['delay'] for v in vs], axis=0),#0 delay
                tf.concat([v['logdelay'] for v in vs], axis=0), #1 logdelay
                tf.concat([v['drops'] for v in vs], axis=0),#2 drops
                tf.concat([v['jitter'] for v in vs], axis=0),#3 jitter
                ]
            )
    
    return tensors

In [22]:
def tfrecord_input_fn(filenames,hparams,shuffle_buf=1000, target='delay'):
    
    files = tf.data.Dataset.from_tensor_slices(filenames)
    files = files.shuffle(len(filenames))

    ds = files.apply(tf.data.experimental.parallel_interleave(
        tf.data.TFRecordDataset, cycle_length=4))

    if shuffle_buf:
        ds = ds.apply(tf.data.experimental.shuffle_and_repeat(shuffle_buf))
    else :
        # sample 10 % for evaluation because it is time consuming
        ds = ds.filter(lambda x: tf.random_uniform(shape=())< 0.1)

    ds = ds.map(lambda buf:parse(buf,target), 
        num_parallel_calls=2)
    ds=ds.prefetch(10)

    it =ds.make_one_shot_iterator()
    sample = transformation_func(it,hparams.batch_size)

    return sample

# Test

In [23]:
hparams = tf.contrib.training.HParams(
    node_count=14,
    link_state_dim=32, 
    #[4, 8, 16, 32, 64]
    path_state_dim=32,
    #[2, 4, 8, 16, 32, 64]
    T=8,
    readout_units=8,
    learning_rate=0.001,
    #[.001, .01, .05]
    batch_size=16,
    #[8, 16, 32, 64]
    dropout_rate=0.5,
    #[.5] leave
    l2=0.1,
    #regulirization constants
    #[.05, .1, .2]
    l2_2=0.01,
    #[.005, .01, .02]
    learn_embedding=True, # If false, only the readout is trained
    readout_layers=2, # number of hidden layers in readout model
    #[2, 3, 4]
)

W0624 12:43:27.897175 140636747016000 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [24]:
testinput = tfrecord_input_fn(train_data, hparams)

W0624 12:43:27.969984 140636747016000 deprecation.py:323] From <ipython-input-22-8f4c204d6b8c>:7: parallel_interleave (from tensorflow.python.data.experimental.ops.interleave_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.interleave(map_func, cycle_length, block_length, num_parallel_calls=tf.data.experimental.AUTOTUNE)` instead. If sloppy execution is desired, use `tf.data.Options.experimental_determinstic`.
W0624 12:43:27.983568 140636747016000 deprecation.py:323] From <ipython-input-22-8f4c204d6b8c>:10: shuffle_and_repeat (from tensorflow.python.data.experimental.ops.shuffle_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.
W0624 12:43:28.008671 140636747016000 deprecation.py:323] From <ipython-input-22-8f4c204d6b8c>:

In [62]:
_r = RouteNet(hparams, output_units=2)
_r.build(input_shape=(8,))
_r.compile(optimizer="adam", loss="mse", metrics=["mae"])
_r.summary()


Model: "route_net_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
edge_update (GRUCell)        multiple                  6240      
_________________________________________________________________
path_update (GRUCell)        multiple                  6240      
_________________________________________________________________
readout (Sequential)         multiple                  336       
_________________________________________________________________
dense_17 (Dense)             multiple                  82        
_________________________________________________________________
mae (MeanAbsoluteError)      multiple                  2         
Total params: 12,900
Trainable params: 12,898
Non-trainable params: 2
_________________________________________________________________


In [63]:
_r.train_step(testinput)['mae']

W0624 13:04:09.044159 140636747016000 optimizer_v2.py:979] Gradients does not exist for variables ['kernel_15:0', 'recurrent_kernel_10:0', 'bias_15:0', 'kernel_16:0', 'recurrent_kernel_11:0', 'bias_16:0'] when minimizing the loss.


<tf.Tensor 'Identity_5:0' shape=() dtype=float32>

In [64]:
np.asarray(testinput[0])

array([<tf.Tensor 'transformation_func/concat:0' shape=(?,) dtype=float32>,
       <tf.Tensor 'transformation_func/concat_1:0' shape=(?,) dtype=float32>,
       <tf.Tensor 'transformation_func/concat_2:0' shape=(?,) dtype=int64>,
       <tf.Tensor 'transformation_func/concat_3:0' shape=(?,) dtype=float32>,
       <tf.Tensor 'transformation_func/concat_4:0' shape=(?,) dtype=int64>,
       <tf.Tensor 'transformation_func/concat_5:0' shape=(?,) dtype=int64>,
       <tf.Tensor 'transformation_func/AddN:0' shape=() dtype=int64>,
       <tf.Tensor 'transformation_func/AddN_1:0' shape=() dtype=int64>,
       <tf.Tensor 'transformation_func/AddN_2:0' shape=() dtype=int64>],
      dtype=object)

In [65]:
_r.fit(testinput, batch_size=None)

ValueError: ('Error when checking model input: expected no data, but got:', ([<tf.Tensor 'transformation_func/concat:0' shape=(?,) dtype=float32>, <tf.Tensor 'transformation_func/concat_1:0' shape=(?,) dtype=float32>, <tf.Tensor 'transformation_func/concat_2:0' shape=(?,) dtype=int64>, <tf.Tensor 'transformation_func/concat_3:0' shape=(?,) dtype=float32>, <tf.Tensor 'transformation_func/concat_4:0' shape=(?,) dtype=int64>, <tf.Tensor 'transformation_func/concat_5:0' shape=(?,) dtype=int64>, <tf.Tensor 'transformation_func/AddN:0' shape=() dtype=int64>, <tf.Tensor 'transformation_func/AddN_1:0' shape=() dtype=int64>, <tf.Tensor 'transformation_func/AddN_2:0' shape=() dtype=int64>], [<tf.Tensor 'transformation_func/concat_6:0' shape=(?,) dtype=float32>, <tf.Tensor 'transformation_func/concat_7:0' shape=(?,) dtype=float32>, <tf.Tensor 'transformation_func/concat_8:0' shape=(?,) dtype=float32>, <tf.Tensor 'transformation_func/concat_9:0' shape=(?,) dtype=float32>]))